In [11]:
import pandas as pd
import numpy as np
import os
import geopandas
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [2]:
measurements_dir = 'data/measurements/'
input_data_dir = os.path.join(measurements_dir, 'input_data.json')
input_data_manual = os.path.join(measurements_dir, "GA_5513_1739_2022_quantitative_status_monitoring_manual.json")
input_data_automatic = os.path.join(measurements_dir, "GA_5513_1739_2022_quantitative_status_monitoring_automatic.json")

In [3]:
def prepare_gdf(gdf):
    df = pd.DataFrame(gdf.drop(columns='geometry'))
    df = df.reset_index()
    df = df.set_index(["date", "name"])
    df = df.drop(columns=["index"])
    df = df.drop(columns=["value", "height"])
    df = df.rename({"normalized_value": "value" }, axis=1)
    return df


In [4]:
manual_gdf = geopandas.read_file(input_data_manual)
automatic_gdf = geopandas.read_file(input_data_automatic)

manual_df = prepare_gdf(manual_gdf)
automatic_df = prepare_gdf(automatic_gdf)

manual_df["value"] = manual_df["value"].fillna(automatic_df["value"])
df = manual_df.reset_index()
df = df.set_index(["date"])
df = df.pivot(columns='name', values='value') 

In [5]:
df.isna().sum()

name
II/112/1        3
II/113/1        0
II/114/1        0
II/115/1     1062
II/116/1     1061
II/131/1       22
II/132/1        0
II/1345/1     159
II/1346/1     159
II/1351/1     232
II/1352/1     209
II/292/1        0
II/297/1        2
II/298/1        0
II/472/1      339
II/922/1      852
II/924/1        0
II/931/1        6
II/932/1     1062
II/936/1     1022
II/940/1        0
II/949/1      227
II/951/1      209
II/952/1      209
II/957/1      647
dtype: int64

In [8]:
df

name,II/112/1,II/113/1,II/114/1,II/115/1,II/116/1,II/131/1,II/132/1,II/1345/1,II/1346/1,II/1351/1,...,II/922/1,II/924/1,II/931/1,II/932/1,II/936/1,II/940/1,II/949/1,II/951/1,II/952/1,II/957/1
date,,,,,,,,,,,,,,,,,,,,,
2001-11-05,242.65,238.04,235.21,238.77,236.50,236.84,237.02,NaN,NaN,NaN,...,NaN,308.82,246.39,241.60,248.02,261.22,NaN,NaN,NaN,NaN
2001-11-12,242.64,238.09,235.12,238.78,236.56,236.74,236.92,NaN,NaN,NaN,...,NaN,308.83,246.38,241.62,248.03,261.19,NaN,NaN,NaN,NaN
2001-11-19,242.64,238.14,235.26,238.80,236.64,236.79,236.97,NaN,NaN,NaN,...,NaN,308.85,246.39,241.62,247.99,261.18,NaN,NaN,NaN,NaN
2001-11-26,242.67,238.20,235.12,238.81,236.72,236.84,236.92,NaN,NaN,NaN,...,NaN,308.86,246.37,241.63,248.00,261.15,NaN,NaN,NaN,NaN
2001-12-03,242.63,238.17,235.04,238.82,236.67,236.79,236.87,NaN,NaN,NaN,...,NaN,308.87,246.35,241.62,248.01,261.40,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10-03,242.18,237.97,234.05,NaN,NaN,235.93,235.39,231.54,241.87,238.96,...,NaN,306.33,245.37,NaN,NaN,273.61,195.32,225.63,218.95,209.01
2022-10-10,242.19,237.95,234.05,NaN,NaN,235.89,235.35,231.55,NaN,238.93,...,NaN,306.35,NaN,NaN,NaN,273.31,NaN,225.60,218.98,208.99
2022-10-17,242.19,237.97,234.01,NaN,NaN,235.84,235.33,231.53,NaN,NaN,...,NaN,306.38,NaN,NaN,NaN,273.36,NaN,225.56,219.03,208.96


In [16]:
station = "II/297/1"

In [13]:
def extract_data_for_station(df, station):
    df2 = df[station]
    df2.index = pd.to_datetime(df2.index)
    df2 = df2.reset_index()

    date = '2002-01-01'
    date = datetime.strptime(date, '%Y-%m-%d')
    end_date = '2021-12-01'
    end_date = datetime.strptime(end_date, '%Y-%m-%d')

    output_df = pd.DataFrame(columns = ["date", "value"])

    while date<end_date:
        idx = df2.date.searchsorted(date)
        s_row = df2.iloc[idx]
        value = 0
        if s_row.date != date:
            if s_row.date < date:
                value = (s_row[station] + df2.iloc[idx+1][station])/2.0
            else:
                value = (s_row[station] + df2.iloc[idx-1][station])/2.0
        else:
            value = s_row[station]
        output_df = output_df.append(pd.Series({"date": date.strftime('%Y-%m-%d'), "value": value}), ignore_index=True)
        date = date + relativedelta(months=1)
    output_df = output_df.sort_values(by="date", ignore_index=True)
    return output_df



In [17]:
output_df = extract_data_for_station(df, station)

/var/folders/4p/ctysll0s0551b50jtg8djbs40000gp/T/ipykernel_824/1726938968.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df = output_df.append(pd.Series({"date": date.strftime('%Y-%m-%d'), "value": value}), ignore_index=True)
/var/folders/4p/ctysll0s0551b50jtg8djbs40000gp/T/ipykernel_824/1726938968.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df = output_df.append(pd.Series({"date": date.strftime('%Y-%m-%d'), "value": value}), ignore_index=True)
/var/folders/4p/ctysll0s0551b50jtg8djbs40000gp/T/ipykernel_824/1726938968.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df = output_df.append(pd.Series({"date": date.strftime('%Y-%m-%d'), "value": value}), ignore_index=True)
/var/folders/4p/

In [18]:
output_df

,date,value
0,2002-01-01,279.595
1,2002-02-01,280.040
2,2002-03-01,280.115
3,2002-04-01,279.850
4,2002-05-01,279.590
...,...,...
234,2021-07-01,279.080
235,2021-08-01,278.885
236,2021-09-01,279.275
237,2021-10-01,279.350
